In [ ]:
from playwright.async_api import async_playwright
import asyncio


async def scrape_tweet(url: str) -> str:
    """
    Scrape a single tweet page for Tweet thread e.g.:
        Return parent tweet text as str
    """
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=False)
        context = await browser.new_context(viewport={"width": 1920, "height": 1080})
        page = await context.new_page()
        await page.goto(url)
        await page.wait_for_selector("[data-testid='tweetText']")
        tweet_text = await page.inner_text("[data-testid='tweetText']")
        return tweet_text


if __name__ == "__main__":
    result = asyncio.run(scrape_tweet("https://x.com/MarkBlenner/status/1813332358831243437"))
    print(result)

Just looked at the speaker and attendee list for the ECI Biochemical Engineering meeting in Dublin next week and I AM PUMPED. 
@omalleylab
 @Brian_Pfleger and Varnika Roy put together a great conference.


In [7]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError
import numpy as np
import nest_asyncio

nest_asyncio.apply()


async def get_all_tweets(username: str):
    url = f"https://x.com/{username}"
    tweets_text = []

    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        # skirting around x making you log in
        await page.goto(url)
        await page.wait_for_timeout(10000)
        await page.goto(url, timeout=3000)

        # Scroll and wait for new tweets to load
        last_count = 0
        num_scrolls = 100
        while num_scrolls > 0:
            # Wait for tweet elements to appear
            try:
                await page.wait_for_selector("article[data-testid='tweet']", timeout=7000)
            except TimeoutError:
                break

            tweet_locator = page.locator("article[data-testid='tweet']")
            count = await tweet_locator.count()
            if count == last_count:
                print('no new tweets loaded; duplicates possible')
            last_count = count

            # Gather tweet text
            for i in range(count):
                text_locator = tweet_locator.nth(i).locator("div[lang]")
                # Using nth(0) because tweets may have multiple text divs
                # or you can combine them if needed
                tweet_text = await text_locator.first.inner_text()
                tweets_text.append(tweet_text)

            # Scroll down to load more
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(3500)
            num_scrolls -= 1
            np.savetxt('found_tweets.txt',tweets_text, delimiter=',')

        await browser.close()

    tweets_text = np.array(tweets_text)
    return tweets_text

if __name__ == "__main__":
    username_to_scrape = "MarkBlenner"  # Replace with any valid username
    all_tweets = asyncio.run(get_all_tweets(username_to_scrape))
    print("Number of tweets found:", len(all_tweets))
    for t in all_tweets:
        print(t, "\n---")

TypeError: Mismatch between array dtype ('<U254') and format specifier ('%.18e')

In [9]:
(96-56)/(109-56) *0.05

0.03773584905660378